In [19]:
import pandas as pd
import os
import numpy as np

import plotly.express as px

from power import power_uncond
from tabulate import  tabulate

In [20]:
tract_panel_in = "../../../data/final/tract_panel.parquet"
table_out = "../../../reports/replication/mde-attend.md"

In [21]:
tract_panel = pd.read_parquet(tract_panel_in)

In [22]:
trt_panel = tract_panel.query("(DNC==1)")

In [23]:
power_stats_base = power_uncond(trt_panel, ['transit','UCMP'], 'rides', 0)[['transit','UCMP','mean','std','mde','mds']]
power_stats_base['mds'] /= 5e4
plot_data = power_stats_base.round(1).set_index(['UCMP','transit']).sort_index()

In [24]:
plot_data.index.set_names(['Near DNC', 'Transit'], inplace=True)
plot_data.columns.set_names('Daily Rides', inplace=True)
plot_data['Mean (SD)'] = plot_data['mean'].astype(str) + plot_data['std'].apply(" ({})".format)
plot_data.drop(columns=['mean','std'], inplace=True)
plot_data.rename(columns={'mde': 'Minimum Detectable Effect',
                          'mds': 'MDE Per Attendee'}, inplace=True)
plot_data = plot_data.iloc[:, [2,0,1]]
plot_data

Daily Rides             Mean (SD)  Minimum Detectable Effect  MDE Per Attendee
Near DNC Transit                                                              
0        bike       181.7 (111.2)                       34.0               0.1
         train    4028.0 (3510.3)                     2007.4               1.0
         uber       470.3 (322.7)                       81.9               0.2
1        bike       177.0 (237.1)                       55.4               0.2
         train    2492.9 (2102.1)                      787.0               0.9
         uber      925.3 (1981.5)                      438.9               1.4

In [26]:
with open(table_out,"w") as f:
    tbl = tabulate(plot_data.reset_index(), 
                   headers=plot_data.reset_index().columns, 
                   showindex=False,
                   tablefmt='github')
    f.write(tbl)
    print(tbl)
!cp $table_out ../../../../eric-mc2-cv/static/uploads/

|   Near DNC | Transit   | Mean (SD)       |   Minimum Detectable Effect |   MDE Per Attendee |
|------------|-----------|-----------------|-----------------------------|--------------------|
|          0 | bike      | 181.7 (111.2)   |                        34   |                0.1 |
|          0 | train     | 4028.0 (3510.3) |                      2007.4 |                1   |
|          0 | uber      | 470.3 (322.7)   |                        81.9 |                0.2 |
|          1 | bike      | 177.0 (237.1)   |                        55.4 |                0.2 |
|          1 | train     | 2492.9 (2102.1) |                       787   |                0.9 |
|          1 | uber      | 925.3 (1981.5)  |                       438.9 |                1.4 |
